# IBM Applied Data Science Capstone Project
## Final Project Notebook

This notebook will be used to write, document, and run all the code for my final capstone project

In [1]:
# Import Key Libraries into the Notebook
import pandas as pd
import numpy as np

Add a classic Print Statement to the Notebook

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Add the required libraries for the project

In [3]:
import requests # library to handle requests
import random # library for random number generation

# module to convert an address into latitude and longitude values
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [12]:
# The code was removed by Watson Studio for sharing.

In [7]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7151482 -74.0156573


## Search Format for Foursquare API


In [5]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


### Define the corresponding URL

In [8]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VCDOEWVG3P3ALVKGAOQIC5MPMCQ01FT5HY35MK1B01SW4TCC&client_secret=IJSCBFL1KMKV3PWK5W4O0JJ0OKNFGRL0HSA22T5LW0UVFVJ4&ll=40.7151482,-74.0156573&v=20180604&query=Italian&radius=500&limit=30'

### Send the GET Request and examine the results

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef2bf2883d66d0312008e8e'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351},
      {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
     'distance': 77,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-15929675

### Get relevant part of JSON and transform it into a pandas dataframe

In [10]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,4fa862b3e4b0ebff2f749f06,225 Murray St,US,New York,United States,77,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,Harry's Italian Pizza Bar,v-1592967538
1,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",False,4f3232e219836c91c7bfde94,63 W Broadway,US,New York,United States,499,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71448400000000...",40.714484,-74.009806,10007,NY,Conca Cucina Italian Restaurant,v-1592967538


### Define information of interest and filter dataframe

In [11]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,US,New York,United States,77,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,US,New York,United States,499,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71448400000000...",40.714484,-74.009806,10007,NY,4f3232e219836c91c7bfde94
